###**Preparing the dataset**

Prepare the dataset using Labelme annotation tool (for Instance segmentation) and LabelImg for object detection.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###**Installing Detectron2 on Colab**

In [2]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp39-cp39-linux_x86_64.whl size=44088 sha256=baedbdfcb76d749dde71ea63cf470e9fabb499ceb45214442a603acbfec80bf7
  Stored in directory: /root/.cache/pip/wheels/68/be/8f/b6c454cd264e0b349b47f8ee00755511f277618af9e5dae20d
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [3]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# After this step it will ask you to restart the runtime, please do it.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61430 sha256=bd0318f3ea5b74ecab34b7b24d8ee9b00ece8a1bc01dd8dba6f190b07551a376
  Stored in directory: /root/.cache/pip/wheels/83/42/02/66178d16e5c44dc26d309931

In [1]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2

###**Register the data to Detectron2 config**

In [2]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [3]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = img_anns["imageHeight"]
        record["width"] = img_anns["imageWidth"]
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
classes = ['jug','easy_foothold','sloper','large_sloper','easy_edge','crimp_edge','edge_large_sloper','crimp','u_edge','easy_sloper','double_sided_jug','pinch_sloper','foothold','double_sided_sloper','edge','curve_edge_sloper','pocket','pinch','edge_sloper','volume','large_moon_edge','platform']

data_path = '/content/drive/MyDrive/Third Year Project/slab_images/'

DatasetCatalog.clear()
MetadataCatalog.clear()

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_train")

###**Training the Detectron2 Instance Segmentation Model**

In [5]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg, LazyConfig
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import transforms as T
from detectron2.data import build_detection_train_loader, DatasetMapper

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
print(len(classes))

22


###**Setup custom Trainer for image augmentation**

In [7]:
class AugmentedTrainer(DefaultTrainer):

    @classmethod
    def build_train_loader(cls, cfg):
      augmentation_list = [
        T.Resize((1300,800)),
        T.RandomBrightness(0.5, 2),
        T.RandomContrast(0.5, 2),
        T.RandomSaturation(0.5, 2),
        T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        T.RandomLighting(0.7),
      ]
      mapper = DatasetMapper(cfg, is_train=True, augmentations=augmentation_list)
      return build_detection_train_loader(cfg, mapper=mapper)

In [8]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = AugmentedTrainer(cfg) 
trainer.resume_or_load(resume=False)

[03/15 17:57:10 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [9]:
trainer.train()

[03/15 17:57:24 d2.engine.train_loop]: Starting training from iteration 0
[03/15 17:58:12 d2.utils.events]:  eta: 2:53:55  iter: 19  total_loss: 5.382  loss_cls: 3.012  loss_box_reg: 0.6643  loss_mask: 0.6927  loss_rpn_cls: 0.8954  loss_rpn_loc: 0.1132  time: 2.3569  data_time: 1.5391  lr: 4.9953e-06  max_mem: 3183M
[03/15 17:58:52 d2.utils.events]:  eta: 2:42:04  iter: 39  total_loss: 4.863  loss_cls: 2.897  loss_box_reg: 0.6693  loss_mask: 0.6906  loss_rpn_cls: 0.5132  loss_rpn_loc: 0.1103  time: 2.1585  data_time: 1.0997  lr: 9.9902e-06  max_mem: 3183M
[03/15 17:59:32 d2.utils.events]:  eta: 2:26:33  iter: 59  total_loss: 4.318  loss_cls: 2.59  loss_box_reg: 0.6874  loss_mask: 0.6864  loss_rpn_cls: 0.1997  loss_rpn_loc: 0.09323  time: 2.1119  data_time: 1.1495  lr: 1.4985e-05  max_mem: 3183M
[03/15 18:00:08 d2.utils.events]:  eta: 2:20:01  iter: 79  total_loss: 3.751  loss_cls: 2.196  loss_box_reg: 0.6668  loss_mask: 0.6798  loss_rpn_cls: 0.1245  loss_rpn_loc: 0.1016  time: 2.0267  

###**Save the trained model and config files**

In [10]:
from detectron2.checkpoint import DetectionCheckpointer

# # Save trained model
checkpointer = DetectionCheckpointer(trainer.model, save_dir="/content/drive/MyDrive/Third Year Project/models/")
checkpointer.save("model_0")  

# Save config file
f = open('/content/drive/MyDrive/Third Year Project/models/model_0_cfg.yaml','w')
f.write(cfg.dump())
f.close()

###**Inference using a Saved Model**

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('/content/drive/MyDrive/Third Year Project/models/model_0_cfg.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # Set threshold for this model
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Third Year Project/models/model_0.pth' # Set path model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 22
predictor = DefaultPredictor(cfg)

###**Inference using the Trained Model**

In [9]:
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/Third Year Project/models/model_0.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = ("holds_test", )
predictor = DefaultPredictor(cfg)

In [16]:
test_dataset_dicts = get_data_dicts(data_path+'test', classes)
train_dataset_dicts = get_data_dicts(data_path+'train', classes)

In [17]:
for d in random.sample(test_dataset_dicts, 2):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    plt.figure(figsize=(25,25))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.